In [1]:
INPUT_PATH = "/content/q_corans.jsonl"
OUTPUT_PATH = "/content/output.jsonl"


In [2]:
import torch
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM

# ======================
# 1. Load model
# ======================

MODEL_NAME = "Qwen/Qwen2-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (ro

In [19]:
def build_support_prompt(question: str) -> str:
    prompt = f"""
You are writing a neutral, encyclopedia-style background paragraph.

Task:
Write general factual background about the main topic of the question.
Write exactly 3 sentences.
Each sentence must be on a new line.

Strict rules:
- Do NOT answer the question
- You may mention the main entity or topic of the question
- Do NOT state or imply the specific attribute being asked about
- Do NOT restate, paraphrase, or semantically mirror any answer option
- Do NOT describe causes, effects, benefits, risks, or consequences
- Do NOT use evaluative or judgmental language
- Do NOT mention debates, beliefs, or opinions
- Use only descriptive, non-decisive facts

Question:
{question}

Write the background paragraph now:
""".strip()

    return prompt


In [17]:
@torch.no_grad()
def generate_text(prompt: str, max_new_tokens: int = 150) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[1]

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        top_p=None,
        repetition_penalty=1.1
    )

    # 只 decode 新生成的 tokens
    new_tokens = output_ids[0][input_len:]
    output_text = tokenizer.decode(
        new_tokens,
        skip_special_tokens=True
    )

    return output_text.strip()


In [5]:
import os
import json
def load_processed_ids(path):
    if not os.path.exists(path):
        return set()

    ids = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                ids.add(json.loads(line)["id"])
            except Exception:
                continue
    return ids

In [23]:
def main():
    from transformers import set_seed
    set_seed(42)

    processed_ids = load_processed_ids(OUTPUT_PATH)

    with open(INPUT_PATH, "r", encoding="utf-8") as fin, \
         open(OUTPUT_PATH, "a", encoding="utf-8") as fout:
        max_items = 3
        count = 0
        for line in fin:
            item = json.loads(line)

            if item["id"] in processed_ids:
                continue   # 已处理，跳过

            question = item["question"]

            # ---- support ----
            support_prompt = build_support_prompt(
            question=question            )
            support = generate_text(support_prompt)


            output_item = {
                "id": item["id"],
                "question": question,
                "support": support,
            }

            fout.write(json.dumps(output_item, ensure_ascii=False) + "\n")
            fout.flush()   # 🔑 立刻写盘，防止中断丢失

            print(f"Processed {item['id']}")
            count += 1
            if count >= max_items:
              break

    print("Done.")


if __name__ == "__main__":
    main()


Processed item_43
Processed item_44
Processed item_45
Done.
